In [1]:
import googlemaps
from datetime import datetime

with open('api_key.txt') as f:
    api_key = f.readline()
    f.close
gmaps = googlemaps.Client(api_key)

In [2]:
# Geocode helper functions

# returns the geocode (json containing details of the location)
def google_get_geocode(location):
    try: 
        geocode_result = gmaps.geocode(location)
        return geocode_result[0]
    except: return None

# returns latitude given the geocode json
def google_get_latitude(geocode):
    try: 
        return geocode['geometry']['location']['lat']
    except: 
        return None

# returns longitude given the geocode json
def google_get_longitude(geocode):
    try: 
        return geocode['geometry']['location']['lng']
    except: 
        return None

# returns address given the geocode json
def google_get_address(geocode):
    try: 
        return geocode['formatted_address']
    except: 
        return None

In [4]:
import pandas as pd
import numpy as np

In [5]:
sm_df = pd.read_csv('data/sm.csv')
ayala_df = pd.read_csv('data/ayala.csv')
robinsons_df = pd.read_csv('data/robinsons.csv')

In [10]:
sm_df['temp_mall_address'] = sm_df['mall_name'] + ', ' +sm_df['mall_address']

In [11]:
geocode_data = []
data_list = [sm_df]

# get geocode, latitude, longitude, and formatted address of each place of occurrence 
# in the accident_df, incident_df, and serious_incident_df tables
for table in data_list:
    table['geocode'] = table.temp_mall_address.apply(google_get_geocode)
    table['latitude'] = table.geocode.apply(google_get_latitude)
    table['longitude'] = table.geocode.apply(google_get_longitude)
    table['formatted_address'] = table.geocode.apply(google_get_address)

In [13]:
del sm_df['temp_mall_address']

In [14]:
import os
directory = 'data'
if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
dict_df = {'sm_with_geocode.csv': sm_df}

for key,val in dict_df.items():
    val.to_csv(os.path.join(directory, key), index=False)